Authors : Jinsu Kim, JunHo Park

ⓒ 2022 CCNets, Inc. All Rights Reserved.

![](https://storage.googleapis.com/kaggle-datasets-images/312121/636393/a5097396fc07cf882d3e0d631b100a36/dataset-cover.jpg?t=2019-08-23-15-00-53)

***

<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 1. Content</i></h1>

<br>

> <h4 style = 'font-family: Times New Roman'>
The Dataset is fully dedicated for the developers who want to train the model on Weather Forecasting for Indian climate.<br><br> This dataset provides data from 1st January 2013 to 24th April 2017 in the city of Delhi, India. <br><br>The 4 parameters here are
meantemp, humidity, wind_speed, meanpressure.
    
  
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 2. About Dataset</i></h1>

<br> 
    
5 columns in the dataset, below is <u>description of features</u>:

*  (1) <b>meantemp</b>: Mean temperature averaged out from multiple 3 hour intervals in a day. 
    
*  (2) <b>humidity</b>: Humidity value for the day (units are grams of water vapor per cubic meter volume of air).
    
    
*  (3) <b>wind_speed</b>: Wind speed measured in kmph.
    
    
*  (4) <b>meanpressure</b>: Pressure reading of weather (measure in atm)
    
    
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 3. Goal of the Notebook</i></h1>
    
> <h4 style = 'font-family: Times New Roman'>
Goal is to train and test model using GPT, Pytorch. <br><br>
    The Target Column used here is <b>meantemp</b>
</h4>

 

https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data

***

<a id="1"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #42c2f5'>1.</b> Import Necessary Libraries </b></h1>

In [1]:
import os
import sys
path_append = "../../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
import pandas  as pd
train_df = pd.read_csv(path_append + '../data/Daily Climate/DailyDelhiClimateTrain.csv')
test_df = pd.read_csv(path_append + '../data/Daily Climate/DailyDelhiClimateTest.csv')
train_df.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


<a id="2"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #4290f5'>2.</b> Modeling: Preprocess </b></h1>

In [3]:
print('Train set \n\n')
train_df.head()

Train set 




,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


In [4]:
from  tools.preprocessing.data_frame import auto_preprocess_dataframe

target_columns = ['meantemp']
df = pd.concat([train_df, test_df], axis=0)
len_train = len(train_df)
df, description = auto_preprocess_dataframe(df, target_columns)

train_df = df[:len_train]
test_df = df[len_train:]

description

,Min,Max,Mean,Std,Null Count,Scaled,Encoded
humidity,-3.433935,2.888945,-4.328179e-16,1.240160,0,Minmax,None
wind_speed,-1.144013,6.446100,9.630388e-02,0.810917,0,Robust,None
meanpressure,-75.954763,500.583698,1.153938e-01,13.151422,0,Robust,None
day_of_year_sin,-0.999991,0.999991,5.124306e-02,0.709811,0,None,EncodedDateTime
day_of_year_cos,-0.999963,1.000000,3.489912e-02,0.702109,0,None,EncodedDateTime
meantemp,6.000000,38.714286,2.522192e+01,7.345014,0,None,None


{'num_features': 5,
 'num_classes': 1,
 'encoded_columns': Index(['day_of_year_sin', 'day_of_year_cos'], dtype='object'),
 'one_hot_encoded_columns': Index([], dtype='object'),
 'encoded_datatime_columns': Index(['day_of_year_sin', 'day_of_year_cos'], dtype='object'),
 'scalers': {'humidity': 'minmax',
  'meanpressure': 'robust',
  'wind_speed': 'robust'}}

In [5]:
import torch
from sklearn.model_selection import train_test_split
from tools.preprocessing.dataset import TemplateDataset

min_seq_len = 8
max_seq_len = 16
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, shuffle=False)
# Prepare the training data
train_df_x = train_df.iloc[:-1, :]  # Select all rows except the last one and all columns (present day's data) including the target column(meantemp)
train_df_y = train_df.iloc[1:, -1:]  # Select all rows starting from the second row and only the last column (next day's data) only for the target column(meantemp)

# Prepare the testing data
test_df_x = test_df.iloc[:-1, :]  # Select all rows except the last one and all columns (present day's data) including the target column(meantemp)
test_df_y = test_df.iloc[1:, -1:]  # Select all rows starting from the second row and only the last column (next day's data) only for the target column(meantemp)

print('train df shape: ', train_df.shape)
print('test df shape: ', test_df.shape)
trainset = TemplateDataset(train_df_x, train_df_y, min_seq_len = min_seq_len, max_seq_len = max_seq_len)
testset = TemplateDataset(test_df_x, test_df_y, min_seq_len = max_seq_len, max_seq_len = max_seq_len)

train df shape:  (1103, 6)
test df shape:  (473, 6)


<a id="3"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #427bf5'>3.</b> Modeling</b></h1>

In [6]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

num_features = description["num_features"]
num_classes = description["num_classes"]

data_config = DataConfig(dataset_name = 'daily-delhi-climate', task_type='regression', obs_shape=[num_features + num_classes], label_size=num_classes)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(model_name = 'gpt')

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False) 

Trainer Name: causal_trainer


ModelParameters Parameters:


,ccnet_config,ccnet_network
0,See details below,gpt



Detailed ccnet_config Configuration:


,ccnet_config_num_layers,ccnet_config_d_model,ccnet_config_dropout,ccnet_config_obs_shape,ccnet_config_reset_pretrained,ccnet_config_network_name,ccnet_config_device,ccnet_config_model_name
0,5,256,0.05,[6],False,gpt,None,gpt


TrainingParameters Parameters:


...

,clip_grad_range,decay_rate_100k,learning_rate,max_grad_norm,scheduler_type
0,None,0.05,0.001,1.0,exponential


AlgorithmParameters Parameters:


,error_function,reset_pretrained
0,mse,False


DataConfig Parameters:


,dataset_name,task_type,obs_shape,label_size,explain_size,show_image_indices
0,daily-delhi-climate,regression,[6],1,2,None


In [7]:
trainer_hub.train(trainset, testset)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junhopark. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (.\..\saved\daily-delhi-climate\causal-learning)... Done. 0.0s


Epochs:   0%|          | 0/400 [00:00<?, ?it/s]

Iterations:   0%|          | 0/17 [00:00<?, ?it/s]

Iterations:   0%|          | 0/17 [00:00<?, ?it/s]

...

Iterations:   0%|          | 0/17 [00:00<?, ?it/s]

Iterations:   0%|          | 0/17 [00:00<?, ?it/s]

[82/400][6/17][Time 7.08]
Unified LR across all optimizers: 0.0009588983465414223
=====================Train Metrics=======================
CCNet:  Three Gpt
Inf: 0.2808	Gen: 0.6020	Rec: 0.4654	E: 0.3643	R: 0.0425	P: 0.7653

mse: 2.6511
mae: 1.1915
rmse: 1.6282
r2: 0.9506

=====================Eval Metrics========================
mse: 2.4197
mae: 1.1837
rmse: 1.5555
r2: 0.9449



Iterations:   0%|          | 0/17 [00:00<?, ?it/s]

Iterations:   0%|          | 0/17 [00:00<?, ?it/s]